In [60]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

NLP From Scratch: Translation with a Sequence to Sequence Network and Attention
===============================================================================

**Author**: [Sean Robertson](https://github.com/spro)

This is the third and final tutorial on doing \"NLP From Scratch\",
where we write our own classes and functions to preprocess the data to
do our NLP modeling tasks. We hope after you complete this tutorial that
you\'ll proceed to learn how [torchtext]{.title-ref} can handle much of
this preprocessing for you in the three tutorials immediately following
this one.

In this project we will be teaching a neural network to translate from
French to English.

``` {.sourceCode .sh}
[KEY: > input, = target, < output]

> il est en train de peindre un tableau .
= he is painting a picture .
< he is painting a picture .

> pourquoi ne pas essayer ce vin delicieux ?
= why not try that delicious wine ?
< why not try that delicious wine ?

> elle n est pas poete mais romanciere .
= she is not a poet but a novelist .
< she not not a poet but a novelist .

> vous etes trop maigre .
= you re too skinny .
< you re all alone .
```

\... to varying degrees of success.

This is made possible by the simple but powerful idea of the [sequence
to sequence network](https://arxiv.org/abs/1409.3215), in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

To improve upon this model we\'ll use an [attention
mechanism](https://arxiv.org/abs/1409.0473), which lets the decoder
learn to focus over a specific range of the input sequence.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-   <https://pytorch.org/> For installation instructions
-   `/beginner/deep_learning_60min_blitz`{.interpreted-text role="doc"}
    to get started with PyTorch in general
-   `/beginner/pytorch_with_examples`{.interpreted-text role="doc"} for
    a wide and deep overview
-   `/beginner/former_torchies_tutorial`{.interpreted-text role="doc"}
    if you are former Lua Torch user

It would also be useful to know about Sequence to Sequence networks and
how they work:

-   [Learning Phrase Representations using RNN Encoder-Decoder for
    Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
-   [Sequence to Sequence Learning with Neural
    Networks](https://arxiv.org/abs/1409.3215)
-   [Neural Machine Translation by Jointly Learning to Align and
    Translate](https://arxiv.org/abs/1409.0473)
-   [A Neural Conversational Model](https://arxiv.org/abs/1506.05869)

You will also find the previous tutorials on
`/intermediate/char_rnn_classification_tutorial`{.interpreted-text
role="doc"} and
`/intermediate/char_rnn_generation_tutorial`{.interpreted-text
role="doc"} helpful as those concepts are very similar to the Encoder
and Decoder models, respectively.

**Requirements**


In [61]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

[This question on Open Data Stack
Exchange](https://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages)
pointed me to the open translation site <https://tatoeba.org/> which has
downloads available at <https://tatoeba.org/eng/downloads> - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: <https://www.manythings.org/anki/>

The English to French pairs are too big to include in the repository, so
download to `data/eng-fra.txt` before continuing. The file is a tab
separated list of translation pairs:

``` {.sourceCode .sh}
I am cold.    J'ai froid.
```

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>Download the data from<a href="https://download.pytorch.org/tutorial/data.zip">here</a>and extract it to the current directory.</p>
</div>


Similar to the character encoding used in the character-level RNN
tutorials, we will be representing each word in a language as a one-hot
vector, or giant vector of zeros except for a single one (at the index
of the word). Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/word-encoding.png)


We\'ll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called `Lang` which has word → index (`word2index`) and index → word
(`index2word`) dictionaries, as well as a count of each word
`word2count` which will be used to replace rare words later.


In [62]:
# These represent the start of a sentence (SOS) and end of a sentence (EOS), with predefined values of 0 and 1 respectively.
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    # Adds a word to word2index and index2word if it doesn't already exist, or increments its count in word2count if it has been encountered before.
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.


In [63]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the `reverse`
flag to reverse the pairs.


In [64]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we\'ll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we\'re filtering to sentences that translate to
the form \"I am\" or \"He is\" etc. (accounting for apostrophes replaced
earlier).


In [65]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# reverse=True because the data is in the form of English to French. But what we should do is French to English Translation
# words are in reverse order. French first then English
# that's why p[1] is checked for english prefixes
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

# Filter pairs using the filterPair function
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-   Read text file and split into lines, split lines into pairs
-   Normalize text, filter by length and content
-   Make word lists from sentences in pairs


In [66]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['je suis humain', 'i am human']


The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A [Sequence to Sequence network](https://arxiv.org/abs/1409.3215), or
seq2seq network, or [Encoder Decoder
network](https://arxiv.org/pdf/1406.1078v3.pdf), is a model consisting
of two RNNs called the encoder and decoder. The encoder reads an input
sequence and outputs a single vector, and the decoder reads that vector
to produce an output sequence.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence `Je ne suis pas le chat noir` →
`I am not the black cat`. Most of the words in the input sentence have a
direct translation in the output sentence, but are in slightly different
orders, e.g. `chat noir` and `black cat`. Because of the `ne/pas`
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the \"meaning\" of the input sequence into a single
vector --- a single point in some N dimensional space of sentences.


The Encoder
===========

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/encoder-network.png)


In [67]:
class EncoderRNN(nn.Module):
    # The size of the input vocabulary, i.e., the number of unique words or tokens.
    # The size of the hidden state vector, which is the internal state of the RNN that gets updated over time as the model processes the sequence.
    
    # hidden_size = 128
    # encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)

    def __init__(self, input_vocabulary_size, hidden_size, dropout_p=0.1):
        # super().__init__() This is the simpler and more modern syntax introduced in Python 3. It works in most cases and is more concise. It implicitly refers to the immediate parent class of the current class (in this case, nn.Module).
        # This is the older, more explicit form that was commonly used in Python 2 and early versions of Python 3. In this syntax, you explicitly pass the current class (EncoderRNN) and the instance of the current class (self) as arguments to super().
        super(EncoderRNN, self).__init__()


        self.hidden_size = hidden_size

        #  The input_vocabulary_size represents the size of the input vocabulary, and hidden_size represents the size of each embedding vector (the dimensionality of the word representation).
        self.embedding = nn.Embedding(input_vocabulary_size, hidden_size)
        
        # First hidden_size (input_vocabulary_size): This refers to the dimensionality of the input features at each time step in the sequence. The GRU expects input vectors at each time step to have this size. In your case, the input to the GRU is coming from an embedding layer, which has already transformed word indices into dense vectors of size hidden_size.
        # Second hidden_size (hidden_size): This represents the dimensionality of the GRU's internal hidden state. The hidden state is what the GRU updates and maintains across time steps as it processes the sequence. The hidden state vector at each time step captures the "memory" of what the GRU has seen so far in the sequence. The hidden state is also the output of the GRU at each time step
        # batch_first=True: This tells the GRU that the first dimension of the input tensor corresponds to the batch size (i.e., the input should have shape [batch_size, sequence_length, hidden_size]).
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)

        
        # During training, dropout randomly sets a fraction of the output units (neurons) in a neural network to zero at each forward pass.
        # This is specified by a parameter p, which in your case is 0.1, meaning 10% of the neurons will be randomly dropped out.
        # This forces the remaining units to learn a more robust and generalized representation of the data.
        # During the test phase (or inference), dropout is disabled, and all neurons are active.
        # With dropout, the network can't rely on any specific neurons, as they might be dropped during any iteration. As a result, the model learns to distribute the information across many neurons, reducing overfitting and improving generalization to new data.
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        # The input (which is a sequence of token indices) is first passed through the embedding layer, converting the indices into dense vectors of size hidden_size. Then, dropout is applied to the embedded vectors to randomly zero out some dimensions for regularization during training.
        embedded = self.dropout(self.embedding(input))
        # output: The output from the GRU for each time step (i.e., for each word in the sequence). It has a shape of [batch_size, sequence_length, hidden_size].
        # hidden: The hidden state after the final time step of the GRU for the last word in the sequence. This hidden state can be used to initialize the hidden state of a decoder in a sequence-to-sequence model.
        output, hidden = self.gru(embedded)
        return output, hidden

The Decoder
===========

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.


Simple Decoder
==============

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string `<SOS>`
token, and the first hidden state is the context vector (the encoder\'s
last hidden state).

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/decoder-network.png)


In [68]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_vocabulary_size):
        super(DecoderRNN, self).__init__()
        # Maps each token in the target vocabulary (of size output_size) to a dense vector of size hidden_size.
        self.embedding = nn.Embedding(output_vocabulary_size, hidden_size)
        # Input size: hidden_size (embedding vector size).
        # Hidden size: hidden_size (dimensionality of GRU's internal state).
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        # Maps the GRU's hidden state to a vector of size output_size (corresponding to the vocabulary).
        # Not present in the encoder
        self.out = nn.Linear(hidden_size, output_vocabulary_size)

    # encoder_outputs: The outputs of the encoder RNN (not used here but may be helpful in attention mechanisms).
    # encoder_hidden: The final hidden state of the encoder RNN (used to initialize the decoder's hidden state).
    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        # Extracts the batch size from encoder_outputs. Each batch contains multiple sequences.
        batch_size = encoder_outputs.size(0)
        # A tensor filled with the start-of-sequence (SOS) token, which initializes the decoding process.
        # Shape: [batch_size, 1] (one token per sequence).
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        # Sets the initial hidden state of the decoder to the final hidden state of the encoder. This links the encoding and decoding processes.
        decoder_hidden = encoder_hidden
        # A list to store the outputs (predictions) for each time step.
        decoder_outputs = []

        # Iterates over the maximum sequence length to decode each token one by one.
        for i in range(MAX_LENGTH):
            # Calls the forward_step method to produce the next token and update the hidden state.
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            # Appends the decoder output to the list of outputs.
            decoder_outputs.append(decoder_output)

            # If the target sequence (target_tensor) is provided, use the actual token from the target as the next input to the decoder.
            # During training
            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                # target_tensor[:, i]: Selects the i-th token for each sequence in the batch.
                # Unsqueeze(1): Adds a dimension to match the shape [batch_size, 1].
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                #  If no target is provided, use the decoder's own predictions as the next input.
                # Without teacher forcing: use its own predictions as the next input

                # decoder_output.topk(1): Selects the index of the highest probability token (predicted token).
                # this is not decoder outputs list!!!!!!!

                # decoder_output is the raw output from the decoder for the current time step. It has shape [batch_size, 1, output_size].
                # output_size is the size of the target vocabulary, and each element represents the unnormalized logit for each token in the vocabulary.
                # .topk(1) retrieves the highest value (the most likely token) along the last dimension (output_size).

                # topk returns two tensors:
                    # Values: The probability scores (not used here).
                    # Indices (topi): The token indices corresponding to the highest probability scores.
                _, topi = decoder_output.topk(1)

                # The topi tensor (indices of the predicted tokens) has shape [batch_size, 1].
                # squeeze(-1) removes the singleton dimension (1) at the last axis, resulting in a tensor of shape [batch_size].
                # Example:
                # Before squeezing: [[2], [4], [0]]
                # After squeezing: [2, 4, 0]

                # .detach():

                # Purpose: Detaches the tensor from the computation graph.
                # Normally, PyTorch keeps track of all operations on tensors to compute gradients during backpropagation. Here, we don't want gradients to flow through the prediction (because it's being used as an input for the next step, not part of the loss computation).
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input
        
        # Combines the outputs for all time steps into a single tensor of shape [batch_size, MAX_LENGTH, output_size].
        # this is the list!!
        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        # Converts raw scores into log-probabilities over the vocabulary for each token. This is useful for loss calculation.
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)

        # Outputs:
        # decoder_outputs: Log-probabilities over the vocabulary for the entire sequence.
        # decoder_hidden: The final hidden state of the decoder.
        # None: Placeholder for compatibility with training loops that expect additional outputs (e.g., attention weights).        
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        # Converts the input token index into its dense embedding vector.
        output = self.embedding(input)
        # Applies a ReLU activation to introduce non-linearity.
        output = F.relu(output)
        #Passes the embedding through the GRU, updating the hidden state.
        output, hidden = self.gru(output, hidden)
        # Maps the GRU output to a vector of size output_size (raw scores over the vocabulary).
        output = self.out(output)
        return output, hidden

I encourage you to train and observe the results of this model, but to
save space we\'ll be going straight for the gold and introducing the
Attention Mechanism.


Attention Decoder
=================

If only the context vector is passed between the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to \"focus\" on a different part of
the encoder\'s outputs for every step of the decoder\'s own outputs.
First we calculate a set of *attention weights*. These will be
multiplied by the encoder output vectors to create a weighted
combination. The result (called `attn_applied` in the code) should
contain information about that specific part of the input sequence, and
thus help the decoder choose the right output words.

![](https://i.imgur.com/1152PYf.png)

Calculating the attention weights is done with another feed-forward
layer `attn`, using the decoder\'s input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/attention-decoder-network.png)

Bahdanau attention, also known as additive attention, is a commonly used
attention mechanism in sequence-to-sequence models, particularly in
neural machine translation tasks. It was introduced by Bahdanau et al.
in their paper titled [Neural Machine Translation by Jointly Learning to
Align and Translate](https://arxiv.org/pdf/1409.0473.pdf). This
attention mechanism employs a learned alignment model to compute
attention scores between the encoder and decoder hidden states. It
utilizes a feed-forward neural network to calculate alignment scores.

However, there are alternative attention mechanisms available, such as
Luong attention, which computes attention scores by taking the dot
product between the decoder hidden state and the encoder hidden states.
It does not involve the non-linear transformation used in Bahdanau
attention.

In this tutorial, we will be using Bahdanau attention. However, it would
be a valuable exercise to explore modifying the attention mechanism to
use Luong attention.


In [69]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>There are other forms of attention that work around the lengthlimitation by using a relative position approach. Read about "localattention" in <a href="https://arxiv.org/abs/1508.04025">Effective Approaches to Attention-based Neural MachineTranslation</a>.</p>
</div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.


In [70]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

Training the Model
==================

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the `<SOS>` token as its first input, and the last hidden state of the
encoder as its first hidden state.

\"Teacher forcing\" is the concept of using the real target outputs as
each next input, instead of using the decoder\'s guess as the next
input. Using teacher forcing causes it to converge faster but [when the
trained network is exploited, it may exhibit
instability](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf).

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation
-intuitively it has learned to represent the output grammar and can
\"pick up\" the meaning once the teacher tells it the first few words,
but it has not properly learned how to create the sentence from the
translation in the first place.

Because of the freedom PyTorch\'s autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
`teacher_forcing_ratio` up to use more of it.


In [71]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

# During training:

# Forward Pass: Compute predictions and loss.
# Backward Pass (.backward()):
    # Computes the gradient ∇L for each parameter.
    # These gradients are stored in the .grad attribute of each parameter.

# Optimizer Step (.step()):
#     Uses the gradients to update parameters:
#     Where η is the learning rate.



# In PyTorch, every trainable parameter (weights, biases) has an associated .grad attribute.
# python
# for param in model.parameters():
#     print(param.grad)  # Prints the gradient for each parameter

# Before calling .backward(), param.grad is None.
# After .backward(), param.grad holds the computed gradients.



# Why Do We Flatten?
# Let’s say we have the following setup:

# Batch size = 2 (we process two sequences at once).
# Sequence length = 3 (each sequence contains 3 time steps).
# Vocabulary size = 4 (our model predicts words from a vocabulary of 4 words).

# Shape: (batch_size=2, sequence_length=3, vocab_size=4)
# decoder_outputs = torch.tensor([
#     [  # First sequence (batch element 1)
#         [2.0, 1.0, 0.1, -1.0],  # Prediction at time step 1
#         [1.2, 2.5, -0.5, 0.0],  # Prediction at time step 2
#         [0.3, -1.0, 2.1, 0.5]   # Prediction at time step 3
#     ],
#     [  # Second sequence (batch element 2)
#         [1.5, 2.3, -0.2, 0.3],  # Prediction at time step 1
#         [2.1, -0.8, 1.0, 1.4],  # Prediction at time step 2
#         [-0.5, 0.8, 1.9, 2.2]   # Prediction at time step 3
#     ]
# ])

# Shape: (batch_size=2, sequence_length=3)
# target_tensor = torch.tensor([
#     [0, 1, 2],  # Correct words for the first sequence
#     [3, 2, 1]   # Correct words for the second sequence
# ])


# After flattening:
# decoder_outputs.view(-1, decoder_outputs.size(-1))  # Becomes (6, 4)
# target_tensor.view(-1)  # Becomes (6,)

# decoder_outputs = torch.tensor([
#     [2.0, 1.0, 0.1, -1.0],  # Time step 1, seq 1
#     [1.2, 2.5, -0.5, 0.0],  # Time step 2, seq 1
#     [0.3, -1.0, 2.1, 0.5],  # Time step 3, seq 1
#     [1.5, 2.3, -0.2, 0.3],  # Time step 1, seq 2
#     [2.1, -0.8, 1.0, 1.4],  # Time step 2, seq 2
#     [-0.5, 0.8, 1.9, 2.2]   # Time step 3, seq 2
# ])

# target_tensor = torch.tensor([0, 1, 2, 3, 2, 1])

# If we directly pass (2, 3, 4) and (2, 3), PyTorch will throw an error because CrossEntropyLoss expects:
# A 2D logits matrix (num_samples, num_classes), not (batch, seq, vocab).
# A 1D target tensor (num_samples,), not (batch, seq).

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)

        # Decoder return decoder_outputs, decoder_hidden, attentions
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.


In [72]:
# import time
# import math

# def asMinutes(s):
#     m = math.floor(s / 60)
#     s -= m * 60
#     return '%dm %ds' % (m, s)

# # returns elapsed time and estimated remaining time
# def timeSince(since, percent):
#     now = time.time()
#     s = now - since
#     es = s / (percent)
#     rs = es - s
#     return 'Elapsed: %s Remaining: %s ' % (asMinutes(s), asMinutes(rs))



########### used tqdm instead ############################

The whole training process looks like this:

-   Start a timer
-   Initialize optimizers and criterion
-   Create set of training pairs
-   Start empty losses array for plotting

Then we call `train` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.


In [73]:
from tqdm import tqdm

def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in tqdm(range(1, n_epochs + 1), desc="Training Progress", unit="epoch"):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('Avg Loss: %.4f' % print_loss_avg)

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            showPlot(plot_losses)

Plotting results
================

Plotting is done with matplotlib, using the array of loss values
`plot_losses` saved while training.


In [74]:
# import matplotlib.pyplot as plt
# plt.switch_backend('agg')
# import matplotlib.ticker as ticker
# import numpy as np

# def showPlot(points):
#     plt.figure()
#     fig, ax = plt.subplots()
#     # this locator puts ticks at regular intervals
#     loc = ticker.MultipleLocator(base=0.2)
#     ax.yaxis.set_major_locator(loc)
#     plt.plot(points)
#     plt.savefig('loss.png')
#     plt.close()


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points, title="Training Loss Over Time"):
    plt.figure(figsize=(8, 5))
    fig, ax = plt.subplots()

    # Apply moving average smoothing for better visualization
    window_size = max(1, len(points) // 20)  # Smooth over 5% of total points
    smoothed_points = np.convolve(points, np.ones(window_size)/window_size, mode='valid')

    ax.plot(points, label="Raw Loss", alpha=0.3)  # Original loss (faded)
    ax.plot(range(window_size-1, len(points)), smoothed_points, label="Smoothed Loss", color="red")

    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.set_title(title)
    ax.legend()
    ax.grid(True, linestyle="--", alpha=0.6)

    # Improve tick visibility
    loc = ticker.MultipleLocator(base=max(len(points) // 10, 1))
    ax.xaxis.set_major_locator(loc)

    plt.savefig('loss.png')
    plt.close()



Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder\'s predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder\'s
attention outputs for display later.


In [75]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:


In [76]:
# def evaluateRandomly(encoder, decoder, n=10):
#     for i in range(n):
#         pair = random.choice(pairs)
#         print('>', pair[0])
#         print('=', pair[1])
#         output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
#         output_sentence = ' '.join(output_words)
#         print('<', output_sentence)
#         print('')


def evaluateRandomly(encoder, decoder, n=10):
    correct_count = 0  # Initialize the counter for correct translations

    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words).replace('<EOS>', '').strip()
        print('<', output_sentence)

        # Check if the model's output matches the target translation
        if output_sentence == pair[1]:
            print("Correct")
            correct_count += 1
        else:
            print("Incorrect")

        print('')  # Empty line for readability

    # Calculate the percentage of correct translations
    accuracy = (correct_count / n) * 100
    print(f"Accuracy: {accuracy:.2f}% ({correct_count}/{n})")


Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we\'ll get
some reasonable results.

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>If you run this notebook you can train, interrupt the kernel,evaluate, and continue training later. Comment out the lines where theencoder and decoder are initialized and run <code>trainIters</code> again.</p>
</div>


In [78]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

print(encoder)
print(decoder)

train(train_dataloader, encoder, decoder, 100, print_every=5, plot_every=5)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
EncoderRNN(
  (embedding): Embedding(4601, 128)
  (gru): GRU(128, 128, batch_first=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
AttnDecoderRNN(
  (embedding): Embedding(2991, 128)
  (attention): BahdanauAttention(
    (Wa): Linear(in_features=128, out_features=128, bias=True)
    (Ua): Linear(in_features=128, out_features=128, bias=True)
    (Va): Linear(in_features=128, out_features=1, bias=True)
  )
  (gru): GRU(256, 128, batch_first=True)
  (out): Linear(in_features=128, out_features=2991, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


Training Progress:   5%|▌         | 5/100 [01:11<23:19, 14.73s/epoch]

Avg Loss: 1.5281


Training Progress:  10%|█         | 10/100 [02:17<20:03, 13.37s/epoch]

Avg Loss: 0.6830


...

<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

<Figure size 800x500 with 0 Axes>

In [58]:
# Define file paths
encoder_path = "encoder.pth"
decoder_path = "decoder.pth"

# Save state dictionaries
torch.save(encoder.state_dict(), encoder_path)
torch.save(decoder.state_dict(), decoder_path)

print(f"Models saved successfully: {encoder_path}, {decoder_path}")


Models saved successfully: encoder.pth, decoder.pth


In [59]:
# Initialize the models again before loading weights
encoder_loaded = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_loaded = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

# Load saved state dictionaries
encoder_loaded.load_state_dict(torch.load(encoder_path))
decoder_loaded.load_state_dict(torch.load(decoder_path))

print("Models loaded successfully!")

encoder_loaded.eval()
decoder_loaded.eval()
evaluateRandomly(encoder, decoder, 200)

Models loaded successfully!
> nous sommes vraiment mariees
= we re really married
< we re really married
Correct

> je me sens seul
= i m lonely
< i m lonely
Correct

> il est meilleur que moi en maths
= he is better than me at math
< he is better than me at math
Correct

> je suis allergique au gluten
= i m allergic to gluten
< i m allergic to gluten
Correct

> elle se plaint toujours de son travail
= she is always complaining of her job
< she is always complaining of her job
Correct

> j en suis impatient
= i m looking forward to that
< i m looking forward to that
Correct

> vous ne regardez pas
= you aren t looking
< you aren t looking aren t you
Incorrect

> je suis une actrice pas un mannequin
= i m an actress not a model
< i m an actress not a model
Correct

> je creve d envie de la revoir
= i am dying to see her again
< i am dying to see her again
Correct

> nous ne sommes pas des flics
= we re not cops
< we re not cops
Correct

> vous etes responsables de ce que vous faites
= y

Set dropout layers to `eval` mode


In [56]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder, 2000)

> quel menteur vous faites !
= you re such a liar
< you re such a liar
Correct

> tu es chanceux d avoir un travail
= you re lucky that you have a job
< you re lucky that you have a job
Correct

> il quitte narita pour hawaii ce soir
= he is leaving narita for hawaii this evening
< he is leaving narita for hawaii this evening
Correct

> heureuse de te voir en un seul morceau !
= i m glad to see you in one piece
< i m happy to see you in one piece
Incorrect

> vous faites erreur a ce propos
= you are mistaken about that
< you are mistaken about that
Correct

> tu ne me soutiens pas beaucoup
= you re not being very supportive
< you re not being very supportive
Correct

> vous etes celui qui m a entrainee
= you re the one who trained me
< you re the one who trained me
Correct

> je ne suis pas ton amie
= i m no friend of yours
< i m not your friend
Incorrect

> il est celibataire
= he is unmarried
< he is unmarried
Correct

> je suis interesse par l anglais
= i am interested in english
< 

Visualizing Attention
=====================

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run `plt.matshow(attentions)` to see attention output
displayed as a matrix. For a better viewing experience we will do the
extra work of adding axes and labels:


In [57]:
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    # plt.show()
    plt.savefig(f'{input_sentence}attention_plot.png')  # Save the plot as a file
    plt.close()  # Close the plot


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluateAndShowAttention('il n est pas aussi grand que son pere')

evaluateAndShowAttention('je suis trop fatigue pour conduire')

evaluateAndShowAttention('je suis desole si c est une question idiote')

evaluateAndShowAttention('je suis reellement fiere de vous')

input = il n est pas aussi grand que son pere
output = he is not as tall as his father <EOS>
input = je suis trop fatigue pour conduire
output = i m too tired to drive <EOS>


C:\Users\shami\AppData\Local\Temp\ipykernel_12332\595099033.py:8: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''] + input_sentence.split(' ') +
C:\Users\shami\AppData\Local\Temp\ipykernel_12332\595099033.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels([''] + output_words)


input = je suis desole si c est une question idiote
output = i m sorry if this is a stupid question <EOS>
input = je suis reellement fiere de vous
output = i m really proud of you <EOS>


Exercises
=========

-   Try with a different dataset
    -   Another language pair
    -   Human → Machine (e.g. IOT commands)
    -   Chat → Response
    -   Question → Answer
-   Replace the embeddings with pretrained word embeddings such as
    `word2vec` or `GloVe`
-   Try with more layers, more hidden units, and more sentences. Compare
    the training time and results.
-   If you use a translation file where pairs have two of the same
    phrase (`I am test \t I am test`), you can use this as an
    autoencoder. Try this:
    -   Train as an autoencoder
    -   Save only the Encoder network
    -   Train a new Decoder for translation from there
